In [1]:
import os
import re
import pickle
from typing import Tuple, List

import spacy
import multiprocessing
#from gensim import 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse
import gensim
from gensim.corpora import Dictionary, MmCorpus
from collections import deque
from collections import namedtuple
import random
from gensim.models.coherencemodel import CoherenceModel
# explore the topics discussed in the reviews and compare how they have evolved over time
# We will define a series of functions to achieve that objective
from gensim.models import phrases
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os

In [ ]:
# read the reviews
pitchfork = pd.read_csv('../data/pitchfork/pitchfork.csv', low_memory=False)
pitchfork.head()

In [ ]:
# read the stopwords
with open('../data/pitchfork/stop.txt', 'r') as f:
    stop_words = f.read().split('\n')
stop_words = stop_words+['songs', 'albums','record','records', 'album', 'sound', 'music',
                        'band', 'song', 'time', 'years',]
stop_words = set(stop_words)

In [ ]:
pitchfork['review'].describe()

In [ ]:
#pitchfork['review'] = pitchfork['review'].values.astype('str')
documents = pitchfork['review'].tolist()
print(len(documents))

In [ ]:
documents = [str(doc) for doc in documents]

In [ ]:
print(len(documents))
documents = [doc for doc in documents if len(doc)>200]
print(len(documents))

In [ ]:
def tokenize(documents: List[str], stop_words: List[str] = None, use_gpu = True, 
             model = 'en_core_web_md') -> List[List[str]]:
    if use_gpu:
        spacy.prefer_gpu()
        print(spacy.prefer_gpu())
    # load the model
    nlp = spacy.load(model, disable=['ner', 'parser'])
    # Mark them as stop words
    for w in stop_words:
        nlp.vocab[w].is_stop = True
    docs = nlp.pipe(documents, batch_size=256,n_process=multiprocessing.cpu_count()-1)
    docs = [[token.lower_ for token in doc if not (token.is_stop or token.is_punct or token.is_digit)] for doc in docs]
    return docs

In [ ]:
# create bigrams and create corpus and dictionary to feed into model
def make_bigrams(texts):
    phrases2 = phrases.Phrases(texts, min_count = 10, threshold = 50)
    bigram = phrases.Phraser(phrases2)
    return [bigram[doc] for doc in texts]

In [ ]:
%%time
documents_tokenized = tokenize(documents, stop_words = stop_words)

In [ ]:
%%time
documents_tokenized = make_bigrams(documents_tokenized)

In [ ]:
# create a dictionary of tokens
dictionary = Dictionary(documents_tokenized)

In [ ]:
# filter extremes and compactify
print(len(dictionary))
dictionary.filter_extremes(no_below = 30, keep_n = 20000)
dictionary.compactify()
print(len(dictionary))

In [ ]:
# we will now create the bow representation
bows, _docs = [],[]
for doc in documents_tokenized:
    _bow = dictionary.doc2bow(doc)
    if _bow!=[]:
        _docs.append(list(doc))
        bows.append(_bow)
docs = _docs

In [ ]:
# inspect a document, it has the token alongside the count of the number of occurrences of that word
#dir(Dictionary)

In [ ]:
# save the corpuss, dictionary and text
gensim.corpora.MmCorpus.serialize('../data/pitchfork/corpus.mm', bows)
dictionary.save_as_text('../data/pitchfork/dict.txt')
with open('../data/pitchfork/dict.pkl','wb') as f:
    pickle.dump(dictionary,f)
with open('../data/pitchfork/docs.pkl','wb') as f:
    pickle.dump(docs,f)
vocab_size = len(dictionary)
num_docs = len(bows)
print(f'Processed {len(bows)} documents.')

In [2]:
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os
#dictionary = Dictionary(
#   simple_preprocess(line, deacc =True) for line in open('../data/pitchfork/dict.txt'))
dictionary = Dictionary.load('../data/pitchfork/dict.pkl')
bows = gensim.corpora.MmCorpus('../data/pitchfork/corpus.mm')
docs = pickle.load(open('../data/pitchfork/docs.pkl','rb'))

In [ ]:
len(dictionary)
#item = list(zip(*bows[0]))
#torch.tensor(list(item[1])).float()

In [3]:
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
from torch import nn


class Data_Processing(object):
    def __init__(self, docs, bows, vocab):
        
        self.docs = docs
        self.bows = bows
        self.vocab = vocab
    
# iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        # create an empty torch object to store
        #expand_array(vocab,)
        bow = np.zeros(len(self.vocab))
        item = list(zip(*self.bows[index])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
        bow[list(item[0])] = list(item[1])
        bow = torch.tensor(bow).float()
        #bow = torch.stack(bow,dim=0)
        txt = self.docs[index]
        return txt, bow
        #return {'text':txt, 'bows':bow}
  
    def __len__(self):
        return len(self.docs)
    
    def collate_fn1(self, batch_data):
        texts, bows = list(zip(*batch_data))
        return texts, torch.stack(bows,dim=0)

batch_size = 512

# create a class to process the traininga and test data
training_data = Data_Processing(docs, bows, dictionary)

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=1,
                                        collate_fn=training_data.collate_fn1)}
dataset_sizes = {'train':len(training_data)}
example = next(iter(dataloaders_dict.get('train')))

In [ ]:
#for index, (bows) in enumerate(dataloaders_dict['train']):
#    print(type(bows[0][1]))
example[1].shape

In [ ]:
# this step creates a random order of the documents index, which will be used to randomly pick between 
# test and training
# randomize documents
np.random.seed(27)
np.random.permutation(10)
#idx_permute = np.random.permutation(num_docs).astype(int)
#print(idx_permute[0:3])

In [ ]:
torch.nn.Module

In [5]:
# define the model
import torch
import torch.nn.functional as F
from torch import nn

class ETM(nn.Module):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,512,2000],dropout=0.0,
                num_topics = 20, vocab_size = 2000, t_hidden_size = 800, emb_dim = 300,  
                embeddings=None, train_embeddings=True, drop=0.0,):
        super(ETM, self).__init__()
        
        self.emb_dim = emb_dim
        self.vocab_size = vocab_size
        self.num_topics = num_topics
        self.t_hidden_size = t_hidden_size
        
        # add the layers that map vocab to first hidden size
        # then add layers from hidden to t_hidden_size
        self.encoder = nn.ModuleDict({
                'enc_1': nn.Linear(vocab_size, t_hidden_size),
                'enc_2': nn.Linear(t_hidden_size, emb_dim)})
        # define mu emb_size to num_topics
        self.fc_mu = nn.Linear(emb_dim,num_topics)
        # define logvar emb_sim to num_topics
        self.fc_logvar = nn.Linear(emb_dim,num_topics)
        
        # decoder part, define layers from num_topics to hidden size
        # define layer from hidden_size to vocab size
        #self.decoder = nn.ModuleDict({
        #        'dec_1': nn.Linear(num_topics, t_hidden_size),
        #        'dec_2': nn.Linear(t_hidden_size, vocab_size)})
        
        #self.latent_dim = num_topics
        # dropout probability
        self.dropout = nn.Dropout(p=drop)
        self.fc1 = nn.Linear(num_topics,num_topics)
        # embeddings layer
        self.rho = nn.Linear(emb_dim,vocab_size)
        # embedding for topics
        self.alpha = nn.Linear(emb_dim,num_topics)
        self.decoder = None
        

    def encode(self, x):
        hid = x
        for i,layer in self.encoder.items():
            hid = F.relu(self.dropout(layer(hid)))
        mu, log_var = self.fc_mu(hid), self.fc_logvar(hid)
        return mu, log_var

    def inference(self,x):
        mu, log_var = self.encode(x)
        theta = torch.softmax(x,dim=1)
        return theta
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self,z):
        wght_dec = self.alpha(self.rho.weight) #[K,V]
        beta = F.softmax(wght_dec,dim=0).transpose(1,0)
        res = torch.mm(z,beta)
        logits = torch.log(res+1e-6)
        return logits
    
    def forward(self, x, collate_fn=None):
        mu, log_var = self.encode(x)
        _theta = self.reparameterize(mu, log_var)
        _theta = self.fc1(_theta) 
        if collate_fn!=None:
            theta = collate_fn(_theta)
        else:
            theta = _theta
        x_reconst = self.decode(theta)
        return x_reconst, mu, log_var

In [7]:
mod = ETM(dropout=0.0,num_topics = 20, vocab_size = len(dictionary), t_hidden_size = 800, emb_dim = 300,  
          embeddings=None, train_embeddings=True, drop=0.0)

mod

ETM(
  (encoder): ModuleDict(
    (enc_1): Linear(in_features=20000, out_features=800, bias=True)
    (enc_2): Linear(in_features=800, out_features=300, bias=True)
  )
  (fc_mu): Linear(in_features=300, out_features=20, bias=True)
  (fc_logvar): Linear(in_features=300, out_features=20, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (rho): Linear(in_features=300, out_features=20000, bias=True)
  (alpha): Linear(in_features=300, out_features=20, bias=True)
)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [ ]:
#model = ETM(device = device, num_topics = 25, vocab_size =len(dictionary), 
#            t_hidden_size = 800, rho_size = 300, 
#            emb_size=300, theta_act='tahn',
#            embeddings=None, train_embeddings=True, enc_drop=0.0, debug_mode=False)

In [ ]:
#model

In [9]:
from torch.optim import Adam
lr = 0.0001
optimizer = Adam(mod.parameters(), lr=lr)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)

In [10]:
#model.forward()
def get_topics(model = None, num_topics = 25, top_n_words=10, vocabulary = None) -> List[str]:
        """
        Gets topics. By default, returns the 10 most relevant terms for each topic.
        Parameters:
        ===
            top_n_words (int): number of top words per topic to return
        Returns:
        ===
            list of str: topic list
        """
        with torch.no_grad():
            topics = []
            gammas = model.get_beta()

            for k in range(num_topics):
                gamma = gammas[k]
                top_words = list(gamma.cpu().numpy().argsort()
                                 [-top_n_words:][::-1])
                topic_words = [vocabulary[a] for a in top_words]
                topics.append(topic_words)

            return topics

In [11]:
def get_topic_diversity(beta, topk = 20):
    num_topics = beta.shape[0]
    list_w = np.zeros((num_topics, topk))
    for k in range(num_topics):
        idx = beta[k,:].argsort()[-topk:][::-1]
        list_w[k,:] = idx
    n_unique = len(np.unique(list_w))
    TD = n_unique / (topk * num_topics)
    print('Topic diveristy is: {}'.format(TD))
    return TD
    
def get_topic_coherence(beta, data, vocab):
    D = len(data) ## number of docs...data is list of documents
    print('D: ', D)
    TC = []
    num_topics = len(beta)
    for k in range(num_topics):
        print('k: {}/{}'.format(k, num_topics))
        top_10 = list(beta[k].argsort()[-11:][::-1])
        top_words = [vocab[a] for a in top_10]
        TC_k = 0
        counter = 0
        for i, word in enumerate(top_10):
            # get D(w_i)
            D_wi = get_document_frequency(data, word)
            j = i + 1
            tmp = 0
            while j < len(top_10) and j > i:
                # get D(w_j) and D(w_i, w_j)
                D_wj, D_wi_wj = get_document_frequency(data, word, top_10[j])
                # get f(w_i, w_j)
                if D_wi_wj == 0:
                    f_wi_wj = -1
                else:
                    f_wi_wj = -1 + ( np.log(D_wi) + np.log(D_wj)  - 2.0 * np.log(D) ) / ( np.log(D_wi_wj) - np.log(D) )
                # update tmp: 
                tmp += f_wi_wj
                j += 1
                counter += 1
            # update TC_k
            TC_k += tmp 
        TC.append(TC_k)
    print('counter: ', counter)
    print('num topics: ', len(TC))
    TC = np.mean(TC) / counter
    print('Topic coherence is: {}'.format(TC))
    return TC


def nearest_neighbors(word, embeddings, vocab, n_most_similar=5):
    vectors = embeddings.data.cpu().numpy()
    index = vocab.index(word)
    query = vectors[index]
    ranks = vectors.dot(query).squeeze()
    denom = query.T.dot(query).squeeze()
    denom = denom * np.sum(vectors**2, 1)
    denom = np.sqrt(denom)
    ranks = ranks / denom
    mostSimilar = []
    [mostSimilar.append(idx) for idx in ranks.argsort()[::-1]]
    nearest_neighbors = mostSimilar[:n_most_similar]
    nearest_neighbors = [vocab[comp] for comp in nearest_neighbors]
    return nearest_neighbors

def get_most_similar_words(model = None, queries=[], vocabulary = None, n_most_similar=5) -> dict:
        """
        Gets the nearest neighborhoring words for a list of tokens. By default, returns the 20 most similar words for each token in 'queries' array.
        Parameters:
        ===
            queries (list of str): words to find similar ones
            n_most_similar (int): number of most similar words to get for each word given in the input. By default is 20
        Returns:
        ===
            dict of (str, list of str): dictionary containing the mapping between query words given and their respective similar words
        """

        model.eval()

        # visualize word embeddings by using V to get nearest neighbors
        with torch.no_grad():
            embeddings = model.rho.weight  # Vocab_size x E
            

            neighbors = {}
            for word in queries:
                neighbors[word] = nearest_neighbors(
                    word,embeddings, vocabulary, n_most_similar)

            return neighbors

In [12]:
import wandb
wandb.init(entity="jlealtru", project="ETM_runs_p")


# WandB – Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config          # Initialize config
config.batch_size = batch_size          # input batch size for training (default: 64)
config.epochs = 1000             # number of epochs to train (default: 10)
config.lr = lr               # learning rate (default: 0.01)
config.no_cuda = False         # disables CUDA training
config.seed = 42               # random seed (default: 42)
config.log_interval = 10     # how many batches to wait before logging training status


config

wandb: Currently logged in as: jlealtru (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 512, 'epochs': 1000, 'lr': 0.0001, 'no_cuda': False, 'seed': 42, 'log_interval': 10}

In [13]:
def training(model = None, epochs = 100, optimizer = None, vocab = None):
        """
        train the model for the given epoch 
        Args:
            epoch ([type]): [description]
        """
        model.train()
        acc_loss = 0
        acc_kl_theta_loss = 0
        cnt = 0
        trainloss_lst, valloss_lst = [], []
        recloss_lst, klloss_lst = [],[]
        
        for i in range(epochs):
            epochloss_lst = []
            optimizer.zero_grad()
            model.zero_grad()
            for index, (bows) in enumerate(dataloaders_dict['train']):
                # add a counter that will register how many examples we have fed to the
                # model
                #cnt+= batch_size
                
                bows = bows[1].to(device)
                bows_recon,mus,log_vars  = model(bows,lambda x:torch.softmax(x,dim=1))
                logsoftmax = torch.log_softmax(bows_recon,dim=1)
                rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                
                kl_div = -0.5 * torch.sum(1+log_vars-mus.pow(2)-log_vars.exp())
                
                loss = rec_loss + kl_div * 1.0
                
                loss.backward()
                optimizer.step()

                trainloss_lst.append(loss.item()/len(bows))
                epochloss_lst.append(loss.item()/len(bows))
                if (i+1) % 2==0:
                    print(f'Epoch {(i+1):>3d}\tIter {(i+1):>4d}\tLoss:{loss.item()/len(bows):<.7f}\tRec Loss:{rec_loss.item()/len(bows):<.7f}\tKL Div:{kl_div.item()/len(bows):<.7f}')

In [14]:
mod.to(device)
#wandb.watch(model, log="all")
training(mod, epochs=config.epochs, optimizer=optimizer, vocab = dictionary)

The number of the indices I am using for the training is 
Epoch   2	Iter    2	Loss:2561.4707031	Rec Loss:2561.4589844	KL Div:0.0117427
Epoch   2	Iter    2	Loss:2568.1462402	Rec Loss:2568.1337891	KL Div:0.0123389
Epoch   2	Iter    2	Loss:2629.4213867	Rec Loss:2629.4074707	KL Div:0.0139473
Epoch   2	Iter    2	Loss:2663.8898926	Rec Loss:2663.8757324	KL Div:0.0142457
Epoch   2	Iter    2	Loss:2583.2407227	Rec Loss:2583.2270508	KL Div:0.0135672
Epoch   2	Iter    2	Loss:2533.3466797	Rec Loss:2533.3337402	KL Div:0.0129592
Epoch   2	Iter    2	Loss:2584.4428711	Rec Loss:2584.4296875	KL Div:0.0132884
Epoch   2	Iter    2	Loss:2572.3264160	Rec Loss:2572.3142090	KL Div:0.0121484
Epoch   2	Iter    2	Loss:2545.8955078	Rec Loss:2545.8837891	KL Div:0.0116975
Epoch   2	Iter    2	Loss:2600.9511719	Rec Loss:2600.9396973	KL Div:0.0114762
Epoch   2	Iter    2	Loss:2584.7548828	Rec Loss:2584.7436523	KL Div:0.0112255
Epoch   2	Iter    2	Loss:2627.0305176	Rec Loss:2627.0192871	KL Div:0.0111296
Epoch   2	Iter    

Epoch   6	Iter    6	Loss:2542.3876953	Rec Loss:2541.9409180	KL Div:0.4468066
Epoch   6	Iter    6	Loss:2631.4916992	Rec Loss:2630.9484863	KL Div:0.5431110
Epoch   6	Iter    6	Loss:2467.3281250	Rec Loss:2466.8227539	KL Div:0.5054851
Epoch   6	Iter    6	Loss:2486.1977539	Rec Loss:2485.6533203	KL Div:0.5443423
Epoch   6	Iter    6	Loss:2544.3139648	Rec Loss:2543.6411133	KL Div:0.6729339
Epoch   6	Iter    6	Loss:2463.7814941	Rec Loss:2463.1074219	KL Div:0.6740521
Epoch   6	Iter    6	Loss:2476.9899902	Rec Loss:2476.2470703	KL Div:0.7428021
Epoch   6	Iter    6	Loss:2466.4562988	Rec Loss:2465.6086426	KL Div:0.8475462
Epoch   6	Iter    6	Loss:2527.3674316	Rec Loss:2526.3537598	KL Div:1.0136597
Epoch   6	Iter    6	Loss:2523.1057129	Rec Loss:2521.9243164	KL Div:1.1813586
Epoch   6	Iter    6	Loss:2517.4543457	Rec Loss:2516.1394043	KL Div:1.3148222
Epoch   6	Iter    6	Loss:2501.4792480	Rec Loss:2500.0317383	KL Div:1.4476181
Epoch   6	Iter    6	Loss:2509.9414062	Rec Loss:2508.2858887	KL Div:1.6555569

Epoch  12	Iter   12	Loss:2416.0827637	Rec Loss:2413.2226562	KL Div:2.8601427
Epoch  12	Iter   12	Loss:2414.6259766	Rec Loss:2411.8076172	KL Div:2.8184462
Epoch  12	Iter   12	Loss:2346.2990723	Rec Loss:2343.6062012	KL Div:2.6928854
Epoch  12	Iter   12	Loss:2382.1582031	Rec Loss:2379.4492188	KL Div:2.7091005
Epoch  12	Iter   12	Loss:2409.8366699	Rec Loss:2406.9897461	KL Div:2.8469613
Epoch  12	Iter   12	Loss:2447.2548828	Rec Loss:2444.2792969	KL Div:2.9756908
Epoch  12	Iter   12	Loss:2328.1901855	Rec Loss:2325.4672852	KL Div:2.7228131
Epoch  12	Iter   12	Loss:2363.0056152	Rec Loss:2360.1062012	KL Div:2.8994603
Epoch  12	Iter   12	Loss:2422.9445801	Rec Loss:2419.8735352	KL Div:3.0709493
Epoch  12	Iter   12	Loss:2387.6159668	Rec Loss:2384.5507812	KL Div:3.0653052
Epoch  12	Iter   12	Loss:2362.5095215	Rec Loss:2359.4155273	KL Div:3.0940070
Epoch  12	Iter   12	Loss:2414.9223633	Rec Loss:2411.6567383	KL Div:3.2655942
Epoch  12	Iter   12	Loss:2432.6481934	Rec Loss:2429.1430664	KL Div:3.5050492

Epoch  16	Iter   16	Loss:2434.9133301	Rec Loss:2429.8154297	KL Div:5.0978742
Epoch  16	Iter   16	Loss:2327.2868652	Rec Loss:2322.4665527	KL Div:4.8203430
Epoch  16	Iter   16	Loss:2318.0141602	Rec Loss:2313.1469727	KL Div:4.8671498
Epoch  16	Iter   16	Loss:2327.5629883	Rec Loss:2322.4042969	KL Div:5.1587477
Epoch  16	Iter   16	Loss:2366.3884277	Rec Loss:2361.0488281	KL Div:5.3396740
Epoch  16	Iter   16	Loss:2367.9179688	Rec Loss:2362.5891113	KL Div:5.3288536
Epoch  16	Iter   16	Loss:2372.2180270	Rec Loss:2366.7622108	KL Div:5.4558476
Epoch  18	Iter   18	Loss:2315.7875977	Rec Loss:2312.8156738	KL Div:2.9720051
Epoch  18	Iter   18	Loss:2409.7409668	Rec Loss:2406.6247559	KL Div:3.1162238
Epoch  18	Iter   18	Loss:2353.4294434	Rec Loss:2350.3710938	KL Div:3.0582392
Epoch  18	Iter   18	Loss:2331.4946289	Rec Loss:2328.4716797	KL Div:3.0229368
Epoch  18	Iter   18	Loss:2298.7866211	Rec Loss:2295.7607422	KL Div:3.0257947
Epoch  18	Iter   18	Loss:2336.9521484	Rec Loss:2333.7387695	KL Div:3.2134185

Epoch  22	Iter   22	Loss:2399.0429688	Rec Loss:2394.1613770	KL Div:4.8816833
Epoch  22	Iter   22	Loss:2336.7233887	Rec Loss:2332.3237305	KL Div:4.3996897
Epoch  22	Iter   22	Loss:2257.4863281	Rec Loss:2253.4741211	KL Div:4.0121474
Epoch  22	Iter   22	Loss:2364.2360840	Rec Loss:2360.2216797	KL Div:4.0144868
Epoch  22	Iter   22	Loss:2413.8666992	Rec Loss:2409.9357910	KL Div:3.9308944
Epoch  22	Iter   22	Loss:2434.7331543	Rec Loss:2430.9331055	KL Div:3.8000681
Epoch  22	Iter   22	Loss:2350.1069336	Rec Loss:2346.6354980	KL Div:3.4713244
Epoch  22	Iter   22	Loss:2350.0446777	Rec Loss:2346.8330078	KL Div:3.2117867
Epoch  22	Iter   22	Loss:2356.6394043	Rec Loss:2353.4526367	KL Div:3.1868014
Epoch  22	Iter   22	Loss:2343.9167480	Rec Loss:2340.9572754	KL Div:2.9595540
Epoch  22	Iter   22	Loss:2296.4614258	Rec Loss:2293.7678223	KL Div:2.6936021
Epoch  22	Iter   22	Loss:2336.3544922	Rec Loss:2333.7409668	KL Div:2.6134355
Epoch  22	Iter   22	Loss:2412.0532227	Rec Loss:2409.5273438	KL Div:2.5257690

...

Epoch 976	Iter  976	Loss:2326.2851562	Rec Loss:2317.3671875	KL Div:8.9180660
Epoch 976	Iter  976	Loss:2320.4226074	Rec Loss:2311.5285645	KL Div:8.8939943
Epoch 976	Iter  976	Loss:2285.6833496	Rec Loss:2276.8103027	KL Div:8.8730488
Epoch 976	Iter  976	Loss:2250.4987793	Rec Loss:2241.7880859	KL Div:8.7106247
Epoch 976	Iter  976	Loss:2350.7387695	Rec Loss:2341.8730469	KL Div:8.8656607
Epoch 976	Iter  976	Loss:2332.4079590	Rec Loss:2323.4492188	KL Div:8.9586830
Epoch 976	Iter  976	Loss:2332.5334473	Rec Loss:2323.6855469	KL Div:8.8479862
Epoch 976	Iter  976	Loss:2347.8798828	Rec Loss:2339.0122070	KL Div:8.8677168
Epoch 976	Iter  976	Loss:2271.0976562	Rec Loss:2262.3623047	KL Div:8.7353516
Epoch 976	Iter  976	Loss:2300.4860840	Rec Loss:2291.5844727	KL Div:8.9016991
Epoch 976	Iter  976	Loss:2209.5964355	Rec Loss:2201.0019531	KL Div:8.5945339
Epoch 976	Iter  976	Loss:2265.1379395	Rec Loss:2256.2421875	KL Div:8.8956547
Epoch 976	Iter  976	Loss:2293.0905762	Rec Loss:2284.3156738	KL Div:8.7748547

Epoch 980	Iter  980	Loss:2355.4113770	Rec Loss:2346.8776855	KL Div:8.5337353
Epoch 980	Iter  980	Loss:2177.0139160	Rec Loss:2168.5778809	KL Div:8.4359474
Epoch 980	Iter  980	Loss:2271.4582520	Rec Loss:2262.6672363	KL Div:8.7910290
Epoch 980	Iter  980	Loss:2260.5554199	Rec Loss:2252.0012207	KL Div:8.5541248
Epoch 980	Iter  980	Loss:2333.3349609	Rec Loss:2324.6831055	KL Div:8.6518288
Epoch 980	Iter  980	Loss:2324.5913086	Rec Loss:2315.8764648	KL Div:8.7149105
Epoch 980	Iter  980	Loss:2358.3206941	Rec Loss:2349.5525386	KL Div:8.7682308
Epoch 982	Iter  982	Loss:2271.0712891	Rec Loss:2262.4858398	KL Div:8.5854759
Epoch 982	Iter  982	Loss:2317.0832520	Rec Loss:2308.5854492	KL Div:8.4979019
Epoch 982	Iter  982	Loss:2214.8129883	Rec Loss:2206.3149414	KL Div:8.4980621
Epoch 982	Iter  982	Loss:2321.1245117	Rec Loss:2312.4936523	KL Div:8.6307945
Epoch 982	Iter  982	Loss:2269.2958984	Rec Loss:2260.8178711	KL Div:8.4779663
Epoch 982	Iter  982	Loss:2281.1623535	Rec Loss:2272.5734863	KL Div:8.5888729

Epoch 986	Iter  986	Loss:2297.1137695	Rec Loss:2288.1276855	KL Div:8.9861517
Epoch 986	Iter  986	Loss:2272.2973633	Rec Loss:2263.4482422	KL Div:8.8491211
Epoch 986	Iter  986	Loss:2291.1267090	Rec Loss:2282.2353516	KL Div:8.8913441
Epoch 986	Iter  986	Loss:2291.3740234	Rec Loss:2282.3125000	KL Div:9.0615406
Epoch 986	Iter  986	Loss:2296.5068359	Rec Loss:2287.5009766	KL Div:9.0058365
Epoch 986	Iter  986	Loss:2306.4479980	Rec Loss:2297.4228516	KL Div:9.0250912
Epoch 986	Iter  986	Loss:2257.2910156	Rec Loss:2248.5224609	KL Div:8.7685833
Epoch 986	Iter  986	Loss:2323.9370117	Rec Loss:2314.9387207	KL Div:8.9983597
Epoch 986	Iter  986	Loss:2301.9765625	Rec Loss:2293.0727539	KL Div:8.9039135
Epoch 986	Iter  986	Loss:2294.1962891	Rec Loss:2285.2714844	KL Div:8.9247341
Epoch 986	Iter  986	Loss:2329.4033203	Rec Loss:2320.5239258	KL Div:8.8794422
Epoch 986	Iter  986	Loss:2312.2272949	Rec Loss:2303.4560547	KL Div:8.7713070
Epoch 986	Iter  986	Loss:2249.2795410	Rec Loss:2240.5532227	KL Div:8.7264166

Epoch 992	Iter  992	Loss:2345.6984863	Rec Loss:2336.6652832	KL Div:9.0330963
Epoch 992	Iter  992	Loss:2273.1403809	Rec Loss:2264.0864258	KL Div:9.0540161
Epoch 992	Iter  992	Loss:2266.5910645	Rec Loss:2257.5449219	KL Div:9.0460968
Epoch 992	Iter  992	Loss:2238.4018555	Rec Loss:2229.5214844	KL Div:8.8803787
Epoch 992	Iter  992	Loss:2268.0817871	Rec Loss:2258.9716797	KL Div:9.1102200
Epoch 992	Iter  992	Loss:2321.4309082	Rec Loss:2312.1965332	KL Div:9.2344265
Epoch 992	Iter  992	Loss:2279.1315918	Rec Loss:2270.0502930	KL Div:9.0814142
Epoch 992	Iter  992	Loss:2238.1206055	Rec Loss:2229.0615234	KL Div:9.0591850
Epoch 992	Iter  992	Loss:2315.2182617	Rec Loss:2306.1367188	KL Div:9.0814867
Epoch 992	Iter  992	Loss:2263.6340332	Rec Loss:2254.5312500	KL Div:9.1026897
Epoch 992	Iter  992	Loss:2338.2053223	Rec Loss:2328.9785156	KL Div:9.2269049
Epoch 992	Iter  992	Loss:2310.8776855	Rec Loss:2301.7287598	KL Div:9.1489954
Epoch 992	Iter  992	Loss:2227.9821777	Rec Loss:2219.1621094	KL Div:8.8201141

Epoch 996	Iter  996	Loss:2259.1892090	Rec Loss:2250.5190430	KL Div:8.6702213
Epoch 996	Iter  996	Loss:2293.2668457	Rec Loss:2284.4365234	KL Div:8.8303585
Epoch 996	Iter  996	Loss:2265.3208008	Rec Loss:2256.6337891	KL Div:8.6870747
Epoch 996	Iter  996	Loss:2340.3928223	Rec Loss:2331.6303711	KL Div:8.7624302
Epoch 996	Iter  996	Loss:2288.9287109	Rec Loss:2280.1989746	KL Div:8.7298279
Epoch 996	Iter  996	Loss:2352.0524902	Rec Loss:2343.3071289	KL Div:8.7453356
Epoch 996	Iter  996	Loss:2320.7060547	Rec Loss:2311.9145508	KL Div:8.7914648
Epoch 996	Iter  996	Loss:2294.1701660	Rec Loss:2285.4675293	KL Div:8.7026777
Epoch 996	Iter  996	Loss:2287.1330566	Rec Loss:2278.4409180	KL Div:8.6920271
Epoch 996	Iter  996	Loss:2285.8168945	Rec Loss:2277.2128906	KL Div:8.6038876
Epoch 996	Iter  996	Loss:2240.1042480	Rec Loss:2231.4624023	KL Div:8.6419220
Epoch 996	Iter  996	Loss:2300.9472656	Rec Loss:2292.2172852	KL Div:8.7300720
Epoch 996	Iter  996	Loss:2260.9914846	Rec Loss:2252.2495180	KL Div:8.7418906

In [ ]:
get_most_similar_words(model = model, queries=['pop','cash'], vocabulary = dictionary, n_most_similar=5)

In [15]:
get_topics(model = mod, num_topics = 25, top_n_words= 5, vocabulary = dictionary)

AttributeError: 'ETM' object has no attribute 'get_beta'

In [ ]:
with torch.no_grad():
    topics = []
    gammas = model.get_beta()
    for k in range(25):
        gamma = gammas[k]
        top_words = list(gamma.cpu().numpy().argsort())
        topic_words = [vocab[a] for a in top_words]
        topics.append(topic_words)

In [ ]:
gammas[3].cpu().numpy().argsort()
[vocab[a] for a in gammas[2].cpu().numpy().argsort()][-10:-1]
topics_words = []
top_words = list(gammas[2].cpu().numpy().argsort()[-5+1:][::-1])
topic_words = [vocab[a].strip() for a in top_words]
topics_words.append(' '.join(topic_words))
topics_words


In [ ]:

def visualize(self, args, vocabulary, show_emb=False):
    Path.cwd().joinpath("results").mkdir(parents=True, exist_ok=True)
    self.eval()
    model_path = Path.home().joinpath("Projects", 
                                        "Personal", 
                                        "balobi_nini", 
                                        'models', 
                                        'embeddings_one_gram_fast_tweets_only').__str__()
    model_gensim = FT_gensim.load(model_path)

        # need to update this .. 
    queries = ['jazz','rock','radiohead']

        ## visualize topics using monte carlo
    with torch.no_grad():
        print('#'*100)
        print('Visualize topics...')
        topics_words = []
        gammas = self.get_beta()
        for k in range(args.num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[-args.num_words+1:][::-1])
            topic_words = [vocabulary[a].strip() for a in top_words]
            topics_words.append(' '.join(topic_words))
            if show_emb:
                ## visualize word embeddings by using V to get nearest neighbors
                print('#'*100)
                print('Visualize word embeddings by using output embedding matrix')
                try:
                    embeddings = model.rho.weight  # Vocab_size x E
                except:
                    embeddings = self.rho         # Vocab_size x E
                neighbors = []
                for word in queries:
                    print('word: {} .. neighbors: {}'.format(
                        word, nearest_neighbors(model_gensim, word)))
                print('#'*100)


In [ ]:
import os
import re
import time
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
import codecs

class VAE(nn.Module):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,1024,2000],dropout=0.0):

        super(VAE, self).__init__()
        self.encoder = nn.ModuleDict({
            f'enc_{i}':nn.Linear(encode_dims[i],encode_dims[i+1]) 
            for i in range(len(encode_dims)-2)
        })
        self.fc_mu = nn.Linear(encode_dims[-2],encode_dims[-1])
        self.fc_logvar = nn.Linear(encode_dims[-2],encode_dims[-1])

        self.decoder = nn.ModuleDict({
            f'dec_{i}':nn.Linear(decode_dims[i],decode_dims[i+1])
            for i in range(len(decode_dims)-1)
        })
        self.latent_dim = encode_dims[-1]
        self.dropout = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(encode_dims[-1],encode_dims[-1])
        
        
    def encode(self, x):
        hid = x
        for i,layer in self.encoder.items():
            hid = F.relu(self.dropout(layer(hid)))
        mu, log_var = self.fc_mu(hid), self.fc_logvar(hid)
        return mu, log_var

    def inference(self,x):
        mu, log_var = self.encode(x)
        theta = torch.softmax(x,dim=1)
        return theta
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        hid = z
        for i,(_,layer) in enumerate(self.decoder.items()):
            hid = layer(hid)
            if i<len(self.decoder)-1:
                hid = F.relu(self.dropout(hid))
        return hid
    
    def forward(self, x, collate_fn=None):
        mu, log_var = self.encode(x)
        _theta = self.reparameterize(mu, log_var)
        _theta = self.fc1(_theta) 
        if collate_fn!=None:
            theta = collate_fn(_theta)
        else:
            theta = _theta
        x_reconst = self.decode(theta)
        return x_reconst, mu, log_var

In [ ]:
class EVAE(VAE):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,1024,2000],dropout=0.0,emb_dim=300):
        super(EVAE,self).__init__(encode_dims=encode_dims,decode_dims=decode_dims,dropout=dropout)
        self.emb_dim = emb_dim
        self.vocab_size = encode_dims[0]
        self.n_topic = encode_dims[-1]
        self.rho = nn.Linear(emb_dim,self.vocab_size)
        self.alpha = nn.Linear(emb_dim,self.n_topic)
        self.decoder = None

    def decode(self,z):
        wght_dec = self.alpha(self.rho.weight) #[K,V]
        beta = F.softmax(wght_dec,dim=0).transpose(1,0)
        res = torch.mm(z,beta)
        logits = torch.log(res+1e-6)
        return logits


class ETM:
    def __init__(self,bow_dim=10000,n_topic=20,taskname=None,device=None,emb_dim=300):
        self.bow_dim = bow_dim
        self.n_topic = n_topic
        self.emb_dim = emb_dim
        #TBD_fc1
        self.vae = EVAE(encode_dims=[bow_dim,1024,512,n_topic],decode_dims=[n_topic,512,bow_dim],dropout=0.0,emb_dim=emb_dim)
        self.device = device
        self.id2token = None
        self.taskname = taskname
        if device!=None:
            self.vae = self.vae.to(device)

    def train(self,train_data,batch_size=256,learning_rate=1e-3,test_data=None,num_epochs=100,is_evaluate=False,log_every=5,beta=1.0,criterion='cross_entropy',ckpt=None):
        self.vae.train()
        self.id2token = {v:k for k,v in train_data.dictionary.token2id.items()}
        data_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=4,collate_fn=train_data.collate_fn)

        optimizer = torch.optim.Adam(self.vae.parameters(),lr=learning_rate)
        #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        if ckpt:
            self.load_model(ckpt["net"])
            optimizer.load_state_dict(ckpt["optimizer"])
            start_epoch = ckpt["epoch"] + 1
        else:
            start_epoch = 0

        trainloss_lst, valloss_lst = [], []
        recloss_lst, klloss_lst = [],[]
        c_v_lst, c_w2v_lst, c_uci_lst, c_npmi_lst, mimno_tc_lst, td_lst = [], [], [], [], [], []
        for epoch in range(start_epoch, num_epochs):
            epochloss_lst = []
            for iter,data in enumerate(data_loader):
                optimizer.zero_grad()

                txts,bows = data
                bows = bows.to(self.device)
                '''
                n_samples = 20
                rec_loss = torch.tensor(0.0).to(self.device)
                for i in range(n_samples):
                    bows_recon,mus,log_vars = self.vae(bows,lambda x:torch.softmax(x,dim=1))
                    
                    logsoftmax = torch.log_softmax(bows_recon,dim=1)
                    _rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                    rec_loss += _rec_loss
                rec_loss = rec_loss / n_samples
                '''
                bows_recon,mus,log_vars = self.vae(bows,lambda x:torch.softmax(x,dim=1))
                if criterion=='cross_entropy':
                    logsoftmax = torch.log_softmax(bows_recon,dim=1)
                    rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                elif criterion=='bce_softmax':
                    rec_loss = F.binary_cross_entropy(torch.softmax(bows_recon,dim=1),bows,reduction='sum')
                elif criterion=='bce_sigmoid':
                    rec_loss = F.binary_cross_entropy(torch.sigmoid(bows_recon),bows,reduction='sum')
                
                kl_div = -0.5 * torch.sum(1+log_vars-mus.pow(2)-log_vars.exp())
                
                loss = rec_loss + kl_div * beta
                
                loss.backward()
                optimizer.step()

                trainloss_lst.append(loss.item()/len(bows))
                epochloss_lst.append(loss.item()/len(bows))
                if (iter+1) % 10==0:
                    print(f'Epoch {(epoch+1):>3d}\tIter {(iter+1):>4d}\tLoss:{loss.item()/len(bows):<.7f}\tRec Loss:{rec_loss.item()/len(bows):<.7f}\tKL Div:{kl_div.item()/len(bows):<.7f}')
            #scheduler.step()
            if (epoch+1) % log_every==0:
                save_name = f'./ckpt/ETM_{self.taskname}_tp{self.n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}_ep{epoch+1}.ckpt'
                checkpoint = {
                    "net": self.vae.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "epoch": epoch,
                    "param": {
                        "bow_dim": self.bow_dim,
                        "n_topic": self.n_topic,
                        "taskname": self.taskname,
                        "emb_dim": self.emb_dim
                    }
                }
                torch.save(checkpoint,save_name)
                # The code lines between this and the next comment lines are duplicated with WLDA.py, consider to simpify them.
                print(f'Epoch {(epoch+1):>3d}\tLoss:{sum(epochloss_lst)/len(epochloss_lst):<.7f}')
                print('\n'.join([str(lst) for lst in self.show_topic_words()]))
                print('='*30)
                smth_pts = smooth_curve(trainloss_lst)
                plt.plot(np.array(range(len(smth_pts)))*log_every,smth_pts)
                plt.xlabel('epochs')
                plt.title('Train Loss')
                plt.savefig('gsm_trainloss.png')
                if test_data!=None:
                    c_v,c_w2v,c_uci,c_npmi,mimno_tc, td = self.evaluate(test_data,calc4each=False)
                    c_v_lst.append(c_v), c_w2v_lst.append(c_w2v), c_uci_lst.append(c_uci),c_npmi_lst.append(c_npmi), mimno_tc_lst.append(mimno_tc), td_lst.append(td)
                save_name = f'./ckpt/ETM_{self.taskname}_tp{self.n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}.ckpt'
                torch.save(self.vae.state_dict(),save_name)
        scrs = {'c_v':c_v_lst,'c_w2v':c_w2v_lst,'c_uci':c_uci_lst,'c_npmi':c_npmi_lst,'mimno_tc':mimno_tc_lst,'td':td_lst}
        '''
        for scr_name,scr_lst in scrs.items():
            plt.cla()
            plt.plot(np.array(range(len(scr_lst)))*log_every,scr_lst)
            plt.savefig(f'wlda_{scr_name}.png')
        '''
        plt.cla()
        for scr_name,scr_lst in scrs.items():
            if scr_name in ['c_v','c_w2v','td']:
                plt.plot(np.array(range(len(scr_lst)))*log_every,scr_lst,label=scr_name)
        plt.title('Topic Coherence')
        plt.xlabel('epochs')
        plt.legend()
        plt.savefig(f'gsm_tc_scores.png')
        # The code lines between this and the last comment lines are duplicated with WLDA.py, consider to simpify them.


    def evaluate(self,test_data,calc4each=False):
        topic_words = self.show_topic_words()
        return evaluate_topic_quality(topic_words, test_data, taskname=self.taskname, calc4each=calc4each)


    def inference_by_bow(self,doc_bow):
        # doc_bow: torch.tensor [vocab_size]; optional: np.array [vocab_size]
        if isinstance(doc_bow,np.ndarray):
            doc_bow = torch.from_numpy(doc_bow)
        doc_bow = doc_bow.reshape(-1,self.bow_dim).to(self.device)
        with torch.no_grad():
            mu,log_var = self.vae.encode(doc_bow)
            mu = self.vae.fc1(mu) 
            theta = F.softmax(mu,dim=1)
            return theta.detach().cpu().squeeze(0).numpy()


    def inference(self, doc_tokenized, dictionary,normalize=True):
        doc_bow = torch.zeros(1,self.bow_dim)
        for token in doc_tokenized:
            try:
                idx = dictionary.token2id[token]
                doc_bow[0][idx] += 1.0
            except:
                print(f'{token} not in the vocabulary.')
        doc_bow = doc_bow.to(self.device)
        with torch.no_grad():
            mu,log_var = self.vae.encode(doc_bow)
            mu = self.vae.fc1(mu)
            if normalize:
                theta = F.softmax(mu,dim=1)
            return theta.detach().cpu().squeeze(0).numpy()

    def get_embed(self,train_data, num=1000):
        self.vae.eval()
        data_loader = DataLoader(train_data, batch_size=512,shuffle=False, num_workers=4, collate_fn=train_data.collate_fn)
        embed_lst = []
        txt_lst = []
        cnt = 0
        for data_batch in data_loader:
            txts, bows = data_batch
            embed = self.inference_by_bow(bows)
            embed_lst.append(embed)
            txt_lst.append(txts)
            cnt += embed.shape[0]
            if cnt>=num:
                break
        embed_lst = np.concatenate(embed_lst,axis=0)[:num]
        txt_lst = np.concatenate(txt_lst,axis=0)[:num]
        return txt_lst, embed_lst

    def get_topic_word_dist(self,normalize=True):
        self.vae.eval()
        with torch.no_grad():
            idxes = torch.eye(self.n_topic).to(self.device)
            word_dist = self.vae.decode(idxes)  # word_dist: [n_topic, vocab.size]
            if normalize:
                word_dist = F.softmax(word_dist,dim=1)
            return word_dist.detach().cpu().numpy()

    def show_topic_words(self,topic_id=None,topK=15, dictionary=None):
        topic_words = []
        idxes = torch.eye(self.n_topic).to(self.device)
        word_dist = self.vae.decode(idxes)
        word_dist = torch.softmax(word_dist,dim=1)
        vals,indices = torch.topk(word_dist,topK,dim=1)
        vals = vals.cpu().tolist()
        indices = indices.cpu().tolist()
        if self.id2token==None and dictionary!=None:
            self.id2token = {v:k for k,v in dictionary.token2id.items()}
        if topic_id==None:
            for i in range(self.n_topic):
                topic_words.append([self.id2token[idx] for idx in indices[i]])
        else:
            topic_words.append([self.id2token[idx] for idx in indices[topic_id]])
        return topic_words

    def load_model(self, model):
        self.vae.load_state_dict(model)



In [ ]:

model = EVAE(encode_dims=[1024,512,256,20],decode_dims=[20,128,768,1024],emb_dim=300)
model

In [ ]:
mod = ETM(dropout=0.0,num_topics = 20, vocab_size = len(dictionary), t_hidden_size = 800, emb_dim = 300,  
          embeddings=None, train_embeddings=True, drop=0.0)

mod